In [85]:
import hmac
import time
import hashlib
import requests
from urllib.parse import urlencode
import pandas as pd
import datetime
import random

from pandas import json_normalize

In [43]:

''' ======  begin of functions, you don't need to touch ====== '''
def hashing(query_string):
    return hmac.new(SECRET.encode('utf-8'), query_string.encode('utf-8'), hashlib.sha256).hexdigest()

def get_timestamp():
    return int(time.time() * 1000)


def dispatch_request(http_method):
    session = requests.Session()
    session.headers.update({
        'Content-Type': 'application/json;charset=utf-8',
        'X-MBX-APIKEY': KEY
    })
    return {
        'GET': session.get,
        'DELETE': session.delete,
        'PUT': session.put,
        'POST': session.post,
    }.get(http_method, 'GET')

# used for sending request requires the signature
def send_signed_request(http_method, url_path, payload={}):
    query_string = urlencode(payload, True)
    
    if query_string:
        query_string = "{}&timestamp={}".format(query_string, get_timestamp())
    else:
        query_string = 'timestamp={}'.format(get_timestamp())

    url = BASE_URL + url_path + '?' + query_string + '&signature=' + hashing(query_string)
    # print("{} {}".format(http_method, url))
    params = {'url': url, 'params': {}}
    response = dispatch_request(http_method)(**params)
    return response.json()

# used for sending public data request
def send_public_request(url_path, payload={}):
    query_string = urlencode(payload, True)
    url = BASE_URL + url_path
    if query_string:
        url = url + '?' + query_string
    # print("{}".format(url))
    # response = dispatch_request('GET')(url=url)
    response = requests.get(url)
    return response.json()

''' ======  end of functions ====== '''

' ======  end of functions ====== '

In [19]:
# for future market
BASE_URL = 'https://api.binance.com' # production base url

In [66]:
url_path = '/api/v3/exchangeInfo'
payload = {
    'symbol':'BNBBTC'
}

response = send_public_request(url_path)

exchangeInfo = json_normalize(response['symbols'])

/Users/cholian/opt/anaconda3/envs/crypto/lib/python3.9/site-packages/cryptography/hazmat/backends/openssl/x509.py:14: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  warnings.warn(


In [47]:
def time_format(TimeString):
    TimeString = datetime.datetime.strptime(TimeString, "%Y-%m-%d %H:%M:%S")
    TimeString = datetime.datetime.timestamp(TimeString)*1000
    
    return str(int(TimeString))

In [75]:
def listed_date(symbol):
    startTime = '2019-01-01 00:00:00'
    startTime = time_format(startTime)

    url_path = '/api/v3/klines'
    payload = {
        'symbol':symbol,
        'interval': '1d',
        'limit':1000
        
    }

    response = send_public_request(url_path,payload)

    data = pd.DataFrame(response)
    #     change the cols
    col = ['Open_time','Open','High','Low','Close','Volume','Close_time','Quote asset volume','Number of trades',
        'Taker buy volume','Taker buy quote asset volume','Ignore']

    data.columns = col
    #       transfer the timestamp into time
    data['Open_time'] = pd.to_datetime(data['Open_time'],unit='ms',utc=True)
    data['Close_time'] = pd.to_datetime(data['Close_time'],unit='ms',utc=True)
    
    date = data.sort_values(by = 'Open_time',ascending=True).Open_time[0].strftime('%Y-%m-%d')
    
    return date

In [94]:
listed_date(exchangeInfo.symbol[0])

'2019-08-08'

In [96]:
exchangeInfo['quoteAsset'].value_counts()

BTC     403
USDT    401
BUSD    331
BNB     266
ETH     204
TRY      71
EUR      45
USDC     41
BRL      35
PAX      34
TUSD     32
BIDR     29
AUD      27
RUB      22
GBP      22
BKRW     11
NGN      10
UAH       7
DAI       5
ZAR       5
IDRT      4
UST       4
USDS      3
USDP      3
TRX       2
XRP       2
BVND      2
VAI       2
GYEN      2
DOGE      1
DOT       1
Name: quoteAsset, dtype: int64

In [103]:
exchangeInfo[exchangeInfo['quoteAsset'].isin(['USDT','BUSD'])].reset_index(drop =True)

,symbol,status,baseAsset,baseAssetPrecision,quoteAsset,quotePrecision,quoteAssetPrecision,baseCommissionPrecision,quoteCommissionPrecision,orderTypes,icebergAllowed,ocoAllowed,quoteOrderQtyMarketAllowed,allowTrailingStop,isSpotTradingAllowed,isMarginTradingAllowed,filters,permissions
0,BTCUSDT,TRADING,BTC,8,USDT,8,8,8,8,"[LIMIT, LIMIT_MAKER, MARKET, STOP_LOSS_LIMIT, ...",True,True,True,False,True,True,"[{'filterType': 'PRICE_FILTER', 'minPrice': '0...","[SPOT, MARGIN]"
1,ETHUSDT,TRADING,ETH,8,USDT,8,8,8,8,"[LIMIT, LIMIT_MAKER, MARKET, STOP_LOSS_LIMIT, ...",True,True,True,False,True,True,"[{'filterType': 'PRICE_FILTER', 'minPrice': '0...","[SPOT, MARGIN]"
2,BNBUSDT,TRADING,BNB,8,USDT,8,8,8,8,"[LIMIT, LIMIT_MAKER, MARKET, STOP_LOSS_LIMIT, ...",True,True,True,False,True,True,"[{'filterType': 'PRICE_FILTER', 'minPrice': '0...","[SPOT, MARGIN]"
3,BCCUSDT,BREAK,BCC,8,USDT,8,8,8,8,"[LIMIT, LIMIT_MAKER, MARKET, STOP_LOSS_LIMIT, ...",True,True,True,False,True,False,"[{'filterType': 'PRICE_FILTER', 'minPrice': '0...",[SPOT]
4,NEOUSDT,TRADING,NEO,8,USDT,8,8,8,8,"[LIMIT, LIMIT_MAKER, MARKET, STOP_LOSS_LIMIT, ...",True,True,True,False,True,True,"[{'filterType': 'PRICE_FILTER', 'minPrice': '0...","[SPOT, MARGIN]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
727,MBLBUSD,TRADING,MBL,8,BUSD,8,8,8,8,"[LIMIT, LIMIT_MAKER, MARKET, STOP_LOSS_LIMIT, ...",True,True,True,False,True,False,"[{'filterType': 'PRICE_FILTER', 'minPrice': '0...",[SPOT]
728,MOBUSDT,TRADING,MOB,8,USDT,8,8,8,8,"[LIMIT, LIMIT_MAKER, MARKET, STOP_LOSS_LIMIT, ...",True,True,True,False,True,False,"[{'filterType': 'PRICE_FILTER', 'minPrice': '0...",[SPOT]
729,MOBBUSD,TRADING,MOB,8,BUSD,8,8,8,8,"[LIMIT, LIMIT_MAKER, MARKET, STOP_LOSS_LIMIT, ...",True,True,True,False,True,False,"[{'filterType': 'PRICE_FILTER', 'minPrice': '0...",[SPOT]
730,NEXOUSDT,TRADING,NEXO,8,USDT,8,8,8,8,"[LIMIT, LIMIT_MAKER, MARKET, STOP_LOSS_LIMIT, ...",True,True,True,False,True,False,"[{'filterType': 'PRICE_FILTER', 'minPrice': '0...",[SPOT]


In [104]:
all_date = []
for i in exchangeInfo.symbol:
    time.sleep(random.randint(1,10)*0.1)
    all_date.append(listed_date(i))

/Users/cholian/opt/anaconda3/envs/crypto/lib/python3.9/site-packages/cryptography/hazmat/backends/openssl/x509.py:14: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  warnings.warn(


ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [91]:
all_date

['2019-08-08',
 '2019-08-08',
 '2019-08-08',
 '2019-08-08',
 '2019-08-08',
 '2019-08-08',
 '2019-08-08',
 '2019-08-08',
 '2017-08-02',
 '2019-08-08',
 '2019-08-08',
 '2019-08-08',
 '2019-08-08',
 '2017-08-18',
 '2017-08-21',
 '2017-08-22',
 '2018-02-07',
 '2017-08-23',
 '2018-01-28',
 '2019-08-08',
 '2018-05-06',
 '2019-08-08',
 '2019-08-08',
 '2019-08-08',
 '2019-06-13',
 '2019-08-08',
 '2019-08-08',
 '2019-08-08',
 '2019-08-08',
 '2018-02-17',
 '2018-02-17',
 '2018-12-01',
 '2017-09-19',
 '2018-11-21',
 '2018-11-21',
 '2019-08-08',
 '2019-08-08',
 '2019-03-17',
 '2019-08-08',
 '2019-08-05',
 '2017-09-27',
 '2019-08-08',
 '2019-08-08',
 '2019-08-08',
 '2019-08-08',
 '2019-08-08',
 '2019-08-08',
 '2019-08-08',
 '2017-09-30',
 '2017-09-30',
 '2019-08-08',
 '2017-10-05',
 '2019-08-08',
 '2019-08-08',
 '2017-10-10',
 '2017-10-10',
 '2019-08-08',
 '2019-08-08',
 '2019-08-08',
 '2019-08-08',
 '2019-06-13',
 '2017-10-13',
 '2018-01-02',
 '2018-01-02',
 '2019-08-08',
 '2019-08-08',
 '2019-08-